# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных:

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Шаг 1. Изучение данных

In [1]:
import pandas as pd # импорт библиотеки pandas
df = pd.read_csv('/datasets/data.csv') # чтение файла с данными и сохранение в df
df.head(10) # получение первых 10 строк таблицы df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [2]:
df.info() # получение общей информации о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.isna().sum() # подсчёт пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

При исследовании таблицы, в столбце  children найдены 2 неккоретных значения:

In [4]:
df['children'].unique() # просмотр уникальных значений

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Чтобы решить, что с этой ошибкой делать, нужно посмотреть сколько людей с такими ошибками.

In [5]:
df.groupby('children')['children'].count()

children
-1        47
 0     14149
 1      4818
 2      2055
 3       330
 4        41
 5         9
 20       76
Name: children, dtype: int64

Данная ошибка встречается очень редко (менее 1%) и в теории можно просто удалить эти строчки, но эти данные нужня для исследования. 

Есть вероятость, что скорее всего была ошибка при заполнении данных. Заменим вручную на правильные.

In [6]:
df['children'] = df['children'].abs() # исправляем значение -1 ребёнок

In [7]:
df['children'] = df['children'].replace(20, 2) # исправляем значениt 20 детей на 2

In [8]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.479721,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,0.755528,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,0.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [9]:
df.isna().mean()

children            0.000000
days_employed       0.100999
dob_years           0.000000
education           0.000000
education_id        0.000000
family_status       0.000000
family_status_id    0.000000
gender              0.000000
income_type         0.000000
debt                0.000000
total_income        0.100999
purpose             0.000000
dtype: float64

**Вывод**

Основываясь на результатах 1 шага, можно сделать следующие выводы:

1. Количество значений в столбцах различается. Значит, в данных есть пропущенные значения;

2. Некорректные данные в столбце days_employed ;

3. Имеются дубликаты в нескольких столбцах;

4. Количество пропусков в столбцах days_employed и total_income одинаковое, стоит проверить гипотезу о том, что попарны эти пропуски или нет.

Возможно, ошибки получены при неправильном заполнении анкеты или при выгрузке данных.

Чтобы двигаться дальше, нужно устранить проблемы в данных.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [10]:
data = df[df['days_employed'].isna() & df['total_income'].isna()]
display(data) # проверяем гипотезу 4 (см. выводы)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости


In [11]:
weight = data.shape[0] / df.shape[0] # оцениваем удельный вес даных, на сколько их доля велика
weight

0.10099883855981417

In [12]:
median_total_income = df['total_income'].median() # получаем средне медианное значение для заполнеие пропусков
median_total_income

145017.93753253992

In [13]:
df['total_income'] = df['total_income'].fillna(value=median_total_income) # заполняем пропуски и проверяем результат
df.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

**Вывод**

Исходя из шага 2.1 можно сделать выводы:

1. Мы убедились в том, что пропуски в столбцах попарно пропущены и из этого снова можно сделать вывод об ошибке при выгрузке данных;

2. Чтобы действовать дальше, нужно было решить, что с этими пропусками делать.

3. Для этого мы выяснили на сколько весомый объём пропусков, он оказался весом (10%), из чего понятно, что удалять такие данные категорически запрещено.

4. Тогда я выбрала путь - заменить пропуски медианными значениями.
 
На этом мой шаг закончен, что делать с пропусками в days_employed, решим дальше, т.к. сначала нужно разобраться с типом данных.

### Замена типа данных

В столбце days_employed отражается общий трудовой стаж в днях, на практике вполне может быть, что нецелочисленное число, но так как тут нужно именно целое количество дня, то переведем столбец в целочисленное значение типа int.

Но сначала нужно разобраться с отрицательными значениями в столбце. Возможно, это ошибка при воде данных, скорее всего, человек, заполнявший данные, ставил "-" перед числовыми значениями. Так, что просто возьмём модуль от числа.

In [14]:
df['days_employed'] = df['days_employed'].abs() # получаем в столбце положительные числа
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Так как данные стобца days_employed не нужны мне для исследования, можно пропуски заменить нули, но доля пропусков значительно больше 1%, нельзя обесценивать эти данные, лучше заменить пропуски на среднемедианное значение.

In [15]:
median_days_employed = df['days_employed'].mean() # получаем средне медианное значение для заполнеие пропусков
median_days_employed

66914.72890682236

In [16]:
df['days_employed'] = df['days_employed'].fillna(value=median_days_employed) # заполняем пропуски и проверяем результат
df.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [17]:
df['total_income'] = df['total_income'].astype(int) # изменяем тип данных на int

In [18]:
df['days_employed'] = df['days_employed'].astype(int) # изменяем тип данных на int
df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529,43,среднее,1,гражданский брак,1,F,компаньон,0,224791,операции с жильем
21521,0,343937,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999,сделка с автомобилем
21522,1,2113,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672,недвижимость
21523,3,3112,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093,на покупку своего автомобиля


**Вывод**

В данном шаге я разобралась  с столбцом days_employed:

1. Сначала мы разобрались с отрицательными значениями
2. Далее заполнили пропуски выбрав наиболее "щадящий" способ
3. Перевели, согласно заданию, столбец days_employed в целочисленный тип данных.

На этом можно сказать, что данные готовы для работы над исследованием.

Но нужно проверить на дубликаты таблицу и обработать их.

### Обработка дубликатов

В столбце education мы видим, что данные заполнены в разном регистре, и нужно это исправить.

In [19]:
df['education'] = df['education'].str.lower() # переводим весь столбец в единый регистр

In [20]:
df['education'].unique() # проверяем результат работы

array(['высшее', 'среднее', 'неоконченное высшее', 'начальное',
       'ученая степень'], dtype=object)

In [21]:
df.duplicated().sum()  # подсчёт явных дубликатов

71

In [22]:
df = df.drop_duplicates().reset_index(drop=True) # удаление явных дубликатов (с удалением старых индексов и формированием новых)

In [23]:
df.duplicated().sum()  # проверка на отсутствие дубликатов

0

**Вывод**

На данном этапе ликвидированы:

неявные дубликаты (значения переведены в единый регистр)

явные дубликаты (удалены идентичные  значения, так как их доля не превышает и 1%)

### Лемматизация

In [24]:
df['purpose'].value_counts() #рассмотрим значения в данном столбце

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

**Вывод**

Анализируя данный столбец, можно выделить 4 категории целей кредита:

- приобретение автомобиля

- приобретение недвижимости

- организация свадьбы

- получение образования

In [25]:
df_unique = df['purpose'].unique() # выведем уникальные значения в виде строки
str_purpose = ' ' #создадим строчки из уникальных значений столбца
for i in df_unique:
    str_purpose += i + ' '
str_purpose

' покупка жилья приобретение автомобиля дополнительное образование сыграть свадьбу операции с жильем образование на проведение свадьбы покупка жилья для семьи покупка недвижимости покупка коммерческой недвижимости покупка жилой недвижимости строительство собственной недвижимости недвижимость строительство недвижимости на покупку подержанного автомобиля на покупку своего автомобиля операции с коммерческой недвижимостью строительство жилой недвижимости жилье операции со своей недвижимостью автомобили заняться образованием сделка с подержанным автомобилем получение образования автомобиль свадьба получение дополнительного образования покупка своего жилья операции с недвижимостью получение высшего образования свой автомобиль сделка с автомобилем профильное образование высшее образование покупка жилья для сдачи на покупку автомобиля ремонт жилью заняться высшим образованием '

Чтобы создать словари для каждой из категорий, необходимо найти леммы

In [26]:
from pymystem3 import Mystem
m = Mystem()


lemmas = m.lemmatize(str_purpose)


from collections import Counter
Counter(lemmas) #для удобства выведем столбцом результат

Counter({' ': 97,
         'покупка': 10,
         'жилье': 7,
         'приобретение': 1,
         'автомобиль': 9,
         'дополнительный': 2,
         'образование': 9,
         'сыграть': 1,
         'свадьба': 3,
         'операция': 4,
         'с': 5,
         'на': 4,
         'проведение': 1,
         'для': 2,
         'семья': 1,
         'недвижимость': 10,
         'коммерческий': 2,
         'жилой': 2,
         'строительство': 3,
         'собственный': 1,
         'подержать': 2,
         'свой': 4,
         'со': 1,
         'заниматься': 2,
         'сделка': 2,
         'получение': 3,
         'высокий': 3,
         'профильный': 1,
         'сдача': 1,
         'ремонт': 1,
         ' \n': 1})

На основе результатов можем выделить словари для каждой категории:

- приобретение автомобиля: 'автомобиль', 'подержать'.

- приобретение недвижимости: 'жилье', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'сдача', 'ремонт'.

- организация свадьбы: 'сыграть', 'свадьба'.

- получение образования: 'дополнительный', 'образование', 'высокий', 'профильный'.

**Вывод**

В данном пункте, проведена лемматизация. Созданы словари для  последующий категоризации.

### Категоризация данных

In [27]:
# создаем словари для категоризации
car = ['автомобиль', 'подержать']
realty = ['жилье', 'недвижимость', 'коммерческий', 'жилой', 'строительство', 'сдача','ремонт']
wedding = ['сыграть', 'свадьба']
education = ['дополнительный', 'образование', 'высокий', 'профильный']

In [28]:
# создадим функцию для категоризации
def new_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    for lemm in lemmas:
        if lemm in car:
            return 'приобретение автомобиля'
        elif lemm in realty:
            return 'приобретение недвижимости'
        elif lemm in wedding:
            return 'организация свадьбы'
        elif lemm in education :
            return 'получение образования'
       

In [29]:
df['new_purpose'] = df['purpose'].apply(new_purpose) # заменияем старые значения на новые точные категории
df['new_purpose'].unique() # проверяем результаты

array(['приобретение недвижимости', 'приобретение автомобиля',
       'получение образования', 'организация свадьбы'], dtype=object)

Проведена категоризация в столбце purpose

Далее проведем  категоризацию в столбце children

In [30]:
def children_quantity(children):
    if children == 0:
        return 'нет детей'
    elif 1 <= children <= 2:
        return 'малодетная семья'
    elif children > 2:
        return 'многодетная семья'

In [31]:
df['children_quantity'] = df['children'].apply(children_quantity) 
df['children_quantity'].unique()

array(['малодетная семья', 'нет детей', 'многодетная семья'], dtype=object)

Проведена категоризация в столбце children

Далее проведем категоризацию в столбце total_income

Выделим 4 категории:

- низший класс (будем определять как отрезок от минимального до четверти от общего дохода)

- стремящиеся к среднему классу (от четверти до медианного значения)

- средний класс (от медианного значения до 3/4 от общего значения)

- высший класс (сверх значения 3/4)

In [32]:
quarter = (df["total_income"]).quantile(0.25)
middle = (df["total_income"].median())
three_quarters = (df["total_income"]).quantile(0.75)

print('нижняя граница:', quarter)

print('серединное значение:' , middle)

print('верхняя граница:', three_quarters)



нижняя граница: 107623.0
серединное значение: 145017.0
верхняя граница: 195813.25


In [33]:
def total_income_category(total_income):
    if total_income < quarter:
        return 'нинизший класс'
    elif quarter <= total_income <= middle:
        return 'стремящиеся к среденему классу'
    elif middle <= total_income <= three_quarters:
        return 'средний класс'
    elif total_income > three_quarters:
        return 'высший класс'

In [34]:
df['total_income_category'] = df['total_income'].apply(total_income_category) 
df['total_income_category'].unique()

array(['высший класс', 'стремящиеся к среденему классу', 'средний класс',
       'нинизший класс'], dtype=object)

Проведена категоризация в столбце total_income

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [35]:
pivot_table_children_quantity = df.pivot_table(index=['children', 'children_quantity'], columns='debt', values='days_employed', aggfunc='count')
pivot_table_children_quantity['percent'] = pivot_table_children_quantity[1] / (pivot_table_children_quantity[1] + pivot_table_children_quantity[0])*100
pivot_table_children_quantity

,debt,0,1,percent
children,children_quantity,,,
0,нет детей,13028.0,1063.0,7.543822
1,малодетная семья,4410.0,445.0,9.165808
2,малодетная семья,1926.0,202.0,9.492481
3,многодетная семья,303.0,27.0,8.181818
4,многодетная семья,37.0,4.0,9.756098
5,многодетная семья,9.0,NaN,NaN


**Вывод**

Основываясь на изучение и сравнении данных можно сделать вывод о том, что количество детей не влияет на появление задолжности по кридиту.

Сначала может показаться, что есть тендеция одновременного повышения количества детей и вероятности появления задолжности, но это не так.

У семьи с 5 детьми, задолжностей не было вообще, в отличие от бездетных семей. Вероятность появления задолжности у семьи с 2 детьми выше, чем с 3.

Что и требывалось даказать.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [36]:
pivot_table_family_status = df.pivot_table(index=['family_status',], columns='debt', values='days_employed', aggfunc='count')
pivot_table_family_status['percent'] = pivot_table_family_status[1] / (pivot_table_family_status[1] + pivot_table_family_status[0])*100
pivot_table_family_status

debt,0,1,percent
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182


**Вывод**

Основываясь на изучении и сравнении данных, можно сделать вывод о том, что семейный статус не влияет на появление задолженности по кредиту.

Самыми благонадежными по данной таблицей являются разведенные и вдовы/вдовцы, т.е. грубо говоря одинокие люди, но при этом же самыми неблагонадежными являются не женатые/замужние.

Наличие мужа/жены тоже не дают однозначных результатов. Женаты/замужние благонадежнее сожителей ("гражданский брак")

Что и требовалось доказать.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [37]:
pivot_table_total_income_category = df.pivot_table(index=['total_income_category',], columns='debt', values='days_employed', aggfunc='count')
pivot_table_total_income_category['percent'] = pivot_table_total_income_category[1] / (pivot_table_total_income_category[1] + pivot_table_total_income_category[0])*100
pivot_table_total_income_category

debt,0,1,percent
total_income_category,,,
высший класс,4981,383,7.140194
нинизший класс,4937,427,7.960477
средний класс,3927,384,8.907446
стремящиеся к среденему классу,5868,547,8.526890


**Вывод**

Основываясь на изучение и сравнении данных, можно сделать вывод о том, что уровень дохода не влияет на появление задолженности по кредиту.

Самыми благонадежными по данной таблицей являются так и высший класс, так и низший класс, что очень неоднозначно и не дает точную картину.

Что и требовалось доказать.


- Как разные цели кредита влияют на его возврат в срок?

In [38]:
pivot_table_new_purpose = df.pivot_table(index=['new_purpose'], columns='debt', values='days_employed', aggfunc='count')
pivot_table_new_purpose['percent'] = pivot_table_new_purpose[1] / (pivot_table_new_purpose[1] + pivot_table_new_purpose[0])*100
pivot_table_new_purpose

debt,0,1,percent
new_purpose,,,
организация свадьбы,2138,186,8.003442
получение образования,3643,370,9.220035
приобретение автомобиля,3903,403,9.359034
приобретение недвижимости,10029,782,7.233373


**Вывод**

Целью, с наименьшей вероятностью появления долга, является приобретение недвижимости. 

На втором месте - организация свадьбы, на третьем - получение образования. 

И целью , с наибольшей вероятностью появления задолженности является приобретение автомобиля.

## Шаг 4. Общий вывод

В результате моего исследования, все 3 гипотезы не подтвердились:
1. Нет зависимости между количеством детей и возврат в срок кредита.

2. Нет зависимости между  семейном положением и возврат в срок кредита.

3. Нет зависимости между уровнем дохода и возврат в срок кредита.

Так же мы рассмотрели, как цели кредита влияют на его возврат в срок.



**Приложение к выводу**

Модно выделить, по данным итогам,  портрет идеального клиента и клиента с самой высокой вероятностью просрочки.

1. Самым идеальным клиентом является бездетный вдова/вдовец с высоким заработком. Объяснить это можно тем, что это связано с тем, что нет затрат на детей, и после смерти супруга/и возможно получения страховки и этим объясняется благонадежность.


2. Клиентом с самой высокой вероятностью просрочки являются, клиент, имеющий детей, не женат/замужем, со среднем заработком. Предположительно, что одинокому родителю тяжелее воспитывать детей, поэтому и высок риск просрочек.